In [1]:
library("data.table")

In [2]:
df <- fread("/mnt/project/Bulk/Genotype Results/Genotype calls/ukb_rel.dat")

In [3]:
get_id_to_drop <- function(tmp){
    tmp[
        , .(ID = c(ID1, ID2))
    ][
        , .(n = .N), by = ID
    ][
        order(-n)[1], ID
    ]
}

In [4]:
# list of all IDs, to mark which ones to drop
ID_list <- df[, .(ID = unique(c(ID1, ID2)))]

In [5]:
# set indicator variable to mark pairs still present
# using the current ID drop set
df[, mask := TRUE]

# initially set all IDs as retained
ID_list[, drop := FALSE]

# iterate IDs from the one with most relatives,
# stop when no pairs remain
while (df[mask == TRUE, .N] > 0) {
    # mask pairs removed by dropping the ID with most relatives
    the_id <- get_id_to_drop(df[mask == TRUE])
    df[ID1 == the_id | ID2 == the_id, mask := FALSE]
    ID_list[ID == the_id, drop := TRUE]
    if ((ID_list[, sum(drop)] %% 1000) == 0) message(
        sprintf(
            "Dropped %s IDs of %s, %s ID pairs remaining",
            ID_list[, sum(drop)],
            nrow(ID_list),
            df[mask == TRUE, .N]
        )
    )
    
    if (df[mask == TRUE, .N] == 0) {
        break
    }
}

Dropped 1000 IDs of 147518, 97322 ID pairs remaining

Dropped 2000 IDs of 147518, 93768 ID pairs remaining

Dropped 3000 IDs of 147518, 90768 ID pairs remaining

Dropped 4000 IDs of 147518, 87768 ID pairs remaining

Dropped 5000 IDs of 147518, 84768 ID pairs remaining

Dropped 6000 IDs of 147518, 82359 ID pairs remaining

Dropped 7000 IDs of 147518, 80359 ID pairs remaining

Dropped 8000 IDs of 147518, 78359 ID pairs remaining

Dropped 9000 IDs of 147518, 76359 ID pairs remaining

Dropped 10000 IDs of 147518, 74359 ID pairs remaining

Dropped 11000 IDs of 147518, 72359 ID pairs remaining

Dropped 12000 IDs of 147518, 70359 ID pairs remaining

Dropped 13000 IDs of 147518, 68359 ID pairs remaining

Dropped 14000 IDs of 147518, 66359 ID pairs remaining

Dropped 15000 IDs of 147518, 64359 ID pairs remaining

Dropped 16000 IDs of 147518, 62359 ID pairs remaining

Dropped 17000 IDs of 147518, 60359 ID pairs remaining

Dropped 18000 IDs of 147518, 58359 ID pairs remaining

Dropped 19000 IDs o

In [21]:
# check if I can include the ID and still have no close relative pairs
# since I drop from the most connected IDs I am not guaranteed that there will not
# be some redundant drops were all the connection of 1 ID have also been dropped
dropped_ids <- ID_list[drop == TRUE, ID]
n_recovered <- 0
n_checked <- 0
n_dropped <- ID_list[, sum(drop)]
for (id in dropped_ids){
    recoverable <- df[
        (
            ((ID1 == id) & !(ID2 %in% dropped_ids)) |
            ((ID2 == id) & !(ID1 %in% dropped_ids))
        ),
        .N == 0
    ]
    if (recoverable) {
        n_recovered <- n_recovered + 1
        ID_list[ID == id, drop := FALSE]
        dropped_ids <- ID_list[drop == TRUE, ID]
    }
    n_checked <- n_checked + 1
    if ((n_checked %% 1000) == 0){
        message(
            sprintf(
                "Recovered %s IDs of %s, %s IDs remaining to check",
                n_recovered,
                n_dropped,
                n_dropped - n_checked
            )
        )
    }
}

Recovered 0 IDs of 74569, 73569 IDs remaining to check

Recovered 0 IDs of 74569, 72569 IDs remaining to check

Recovered 0 IDs of 74569, 71569 IDs remaining to check

Recovered 0 IDs of 74569, 70569 IDs remaining to check

Recovered 0 IDs of 74569, 69569 IDs remaining to check

Recovered 0 IDs of 74569, 68569 IDs remaining to check

Recovered 1 IDs of 74569, 67569 IDs remaining to check

Recovered 1 IDs of 74569, 66569 IDs remaining to check

Recovered 1 IDs of 74569, 65569 IDs remaining to check

Recovered 1 IDs of 74569, 64569 IDs remaining to check

Recovered 1 IDs of 74569, 63569 IDs remaining to check

Recovered 1 IDs of 74569, 62569 IDs remaining to check

Recovered 1 IDs of 74569, 61569 IDs remaining to check

Recovered 1 IDs of 74569, 60569 IDs remaining to check

Recovered 1 IDs of 74569, 59569 IDs remaining to check

Recovered 1 IDs of 74569, 58569 IDs remaining to check

Recovered 1 IDs of 74569, 57569 IDs remaining to check

Recovered 1 IDs of 74569, 56569 IDs remaining to

In [23]:
ids_to_drop <- ID_list[drop == TRUE, .(ID)]

In [24]:
# no pair remains when I remove pairs containing IDs in ids_to_drop
stopifnot(df[!(ID1 %in% ids_to_drop[["ID"]] | ID2 %in% ids_to_drop[["ID"]]), .N] == 0)

In [27]:
# no ID should be recoverable anymore
dropped_ids <- ID_list[drop == TRUE, ID]
n_checked <- 0
n_dropped <- ID_list[, sum(drop)]
for (id in dropped_ids){
    recoverable <- df[
        (
            ((ID1 == id) & !(ID2 %in% dropped_ids)) |
            ((ID2 == id) & !(ID1 %in% dropped_ids))
        ),
        .N == 0
    ]
    stopifnot(!recoverable)
    n_checked <- n_checked + 1
    if ((n_checked %% 1000) == 0){
        message(
            sprintf(
                "Checked %s IDs of %s",
                n_checked,
                n_dropped
            )
        )
    }
}

Checked 1000 IDs of 74566

Checked 2000 IDs of 74566

Checked 3000 IDs of 74566

Checked 4000 IDs of 74566

Checked 5000 IDs of 74566

Checked 6000 IDs of 74566

Checked 7000 IDs of 74566

Checked 8000 IDs of 74566

Checked 9000 IDs of 74566

Checked 10000 IDs of 74566

Checked 11000 IDs of 74566

Checked 12000 IDs of 74566

Checked 13000 IDs of 74566

Checked 14000 IDs of 74566

Checked 15000 IDs of 74566

Checked 16000 IDs of 74566

Checked 17000 IDs of 74566

Checked 18000 IDs of 74566

Checked 19000 IDs of 74566

Checked 20000 IDs of 74566

Checked 21000 IDs of 74566

Checked 22000 IDs of 74566

Checked 23000 IDs of 74566

Checked 24000 IDs of 74566

Checked 25000 IDs of 74566

Checked 26000 IDs of 74566

Checked 27000 IDs of 74566

Checked 28000 IDs of 74566

Checked 29000 IDs of 74566

Checked 30000 IDs of 74566

Checked 31000 IDs of 74566

Checked 32000 IDs of 74566

Checked 33000 IDs of 74566

Checked 34000 IDs of 74566

Checked 35000 IDs of 74566

Checked 36000 IDs of 74566

C

In [29]:
ids_to_drop[, .N]

[1] 74566

In [30]:
fwrite(ids_to_drop, "ids_to_drop_close_relatives.txt")

In [31]:
system("dx upload ids_to_drop_close_relatives.txt --path genotypes/")